# Web Server Log Analysis - Python Take-Home Assessment

## Overview
This assessment involves analyzing the Calgary HTTP dataset, which contains approximately one year's worth of HTTP requests to the University of Calgary's Computer Science web server. You'll work with real-world web server log data to extract meaningful insights and demonstrate your Python data analysis skills.

## Part 1: Data Loading and Cleaning

### Instructions

* Work in the cells below - You can add as many cells as needed for data loading, cleaning, and exploration
* Import required libraries
* Implement data loading and cleaning - Create functions to download, parse, and clean the log data
* Explore the data - Understand the structure and identify any data quality issues

In [36]:
#Step 1: Parsing the Full Log File into a Clean DataFrame

import gzip
import re
from datetime import datetime
import pandas as pd

# Load data
file_path = r"C:\Users\Arshad\Downloads\calgary_access_log.gz"

# Read and parse the file
log_entries = []
with gzip.open(file_path, 'rt', encoding='utf-8', errors='ignore') as f:
    for line in f:
        log_entries.append(line.strip())
print(f"Total lines loaded: {len(log_entries)}")

# Define the log parsing function
log_pattern = re.compile(
    r'^(?P<host>\S+) (?P<rfc931>\S+) (?P<authuser>\S+) \[(?P<datetime>[^\]]+)\] '
    r'"(?P<request>[^"]*)" (?P<status>\d{3}) (?P<bytes>\S+)$'
)

def parse_log_line(line):
    match = log_pattern.match(line)
    if not match:
        return None
    data = match.groupdict()
    try:
        data['datetime'] = datetime.strptime(data['datetime'], "%d/%b/%Y:%H:%M:%S %z")
    except ValueError:
        return None
    data['method'], data['filename'], data['protocol'] = ('', '', '')
    if data['request']:
        parts = data['request'].split()
        if len(parts) == 3:
            data['method'], data['filename'], data['protocol'] = parts
    try:
        data['status'] = int(data['status'])
    except:
        data['status'] = None
    data['bytes'] = int(data['bytes']) if data['bytes'].isdigit() else 0
    data['extension'] = data['filename'].split('.')[-1] if '.' in data['filename'] else ''
    return data

parsed_data = [parse_log_line(line) for line in log_entries]
cleaned_data = [entry for entry in parsed_data if entry]

# Convert to DataFrame
df = pd.DataFrame(cleaned_data)
print(df.head())
print(df.info())


Total lines loaded: 726739
    host rfc931 authuser                   datetime                  request  \
0  local      -        -  1994-10-24 13:41:41-06:00  GET index.html HTTP/1.0   
1  local      -        -  1994-10-24 13:41:41-06:00       GET 1.gif HTTP/1.0   
2  local      -        -  1994-10-24 13:43:13-06:00  GET index.html HTTP/1.0   
3  local      -        -  1994-10-24 13:43:14-06:00       GET 2.gif HTTP/1.0   
4  local      -        -  1994-10-24 13:43:15-06:00       GET 3.gif HTTP/1.0   

   status  bytes method    filename  protocol extension  
0     200    150    GET  index.html  HTTP/1.0      html  
1     200   1210    GET       1.gif  HTTP/1.0       gif  
2     200   3185    GET  index.html  HTTP/1.0      html  
3     200   2555    GET       2.gif  HTTP/1.0       gif  
4     200  36403    GET       3.gif  HTTP/1.0       gif  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 724836 entries, 0 to 724835
Data columns (total 11 columns):
 #   Column     Non-Null Count   

## ⚠️ IMPORTANT: Template Questions Section
**DO NOT MODIFY THE TEMPLATE BELOW THIS POINT**

The following section contains the assessment questions. You may add cells above this section for data loading, cleaning, and exploration, but do not modify the function signatures or structure of the questions below.

## Part 2: Analysis Questions

### Instructions

* Implement each function according to its docstring specifications
* Use the cleaned data you prepared in Part 1
* Ensure your functions return the exact data types specified
* Test your functions to verify they work correctly
* You may add helper functions, but keep the main function signatures unchanged

### Q1: Count of total log records

In [37]:
def total_log_records() -> int:
    """
    Q1: Count of total log records.

    Objective:
        Determine the total number of HTTP log entries in the dataset.
        Each line in the log file represents one HTTP request.

    Returns:
        int: Total number of log entries.
    """

    # TODO: Implement logic to count log records

    return len(df)


answer1 = total_log_records()
print("Answer 1:")
print(answer1)

Answer 1:
724836


### Q2: Count of unique hosts

In [38]:
def unique_host_count() -> int:
    """
    Q2: Count of unique hosts.

    Objective:
        Determine how many distinct hosts accessed the server.

    Returns:
        int: Number of unique hosts.
    """

    # TODO: Implement logic to count unique hosts

    return df['host'].nunique()


answer2 = unique_host_count()
print("Answer 2:")
print(answer2)

Answer 2:
2


### Q3: Date-wise unique filename counts

In [41]:
def datewise_unique_filename_counts() -> dict[str, int]:
    """
    Q3: Date-wise unique filename counts.

    Objective:
        For each date, count the number of unique filenames that accessed the server.
        The date should be in 'dd-MMM-yyyy' format (e.g., '01-Jul-1995').

    Returns:
        dict: A dictionary mapping each date to its count of unique filenames.
              Example: {'01-Jul-1995': 123, '02-Jul-1995': 150}
    """

    # TODO: Implement logic for date-wise unique filename counts

 # Ensure datetime column is in datetime format
    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')

    # Drop rows with invalid datetime
    df_valid = df.dropna(subset=['datetime']).copy()

    # Use .loc to avoid SettingWithCopyWarning
    df_valid.loc[:, 'date_str'] = df_valid['datetime'].dt.strftime('%d-%b-%Y')

    # Group by formatted date and count unique filenames
    result = df_valid.groupby('date_str')['filename'].nunique().to_dict()

    return result


answer3 = datewise_unique_filename_counts()
print("Answer 3:")
print(answer3)

Answer 3:
{'01-Aug-1995': 670, '01-Jul-1995': 388, '01-Jun-1995': 591, '01-May-1995': 467, '01-Oct-1995': 553, '01-Sep-1995': 329, '02-Apr-1995': 439, '02-Aug-1995': 856, '02-Jul-1995': 398, '02-Jun-1995': 514, '02-May-1995': 702, '02-Oct-1995': 872, '02-Sep-1995': 350, '03-Apr-1995': 796, '03-Aug-1995': 583, '03-Jul-1995': 434, '03-Jun-1995': 399, '03-May-1995': 590, '03-Oct-1995': 847, '03-Sep-1995': 213, '04-Apr-1995': 822, '04-Aug-1995': 716, '04-Jul-1995': 611, '04-Jun-1995': 354, '04-May-1995': 685, '04-Oct-1995': 890, '04-Sep-1995': 341, '05-Apr-1995': 891, '05-Aug-1995': 508, '05-Jul-1995': 608, '05-Jun-1995': 495, '05-May-1995': 609, '05-Oct-1995': 847, '05-Sep-1995': 412, '06-Apr-1995': 679, '06-Aug-1995': 449, '06-Jul-1995': 523, '06-Jun-1995': 663, '06-May-1995': 517, '06-Oct-1995': 869, '06-Sep-1995': 550, '07-Apr-1995': 777, '07-Aug-1995': 609, '07-Jul-1995': 428, '07-Jun-1995': 486, '07-May-1995': 726, '07-Oct-1995': 468, '07-Sep-1995': 591, '08-Apr-1995': 543, '08-Aug-1

### Q4: Number of 404 response codes

In [42]:
def count_404_errors() -> int:
    """
    Q4: Number of 404 response codes.

    Objective:
        Count how many times the HTTP 404 Not Found status appears in the logs.

    Returns:
        int: Number of 404 errors.
    """

    # TODO: Implement logic to count 404 errors

    return (df['status'] == 404).sum()


answer4 = count_404_errors()
print("Answer 4:")
print(answer4)

Answer 4:
23517


### Q5: Top 15 filenames with 404 responses

In [43]:
def top_15_filenames_with_404() -> list[tuple[str, int]]:
    """
    Q5: Top 15 filenames with 404 responses.

    Objective:
        Identify which requested URLs most frequently resulted in a 404 error.
        Return the top 15 filenames sorted by frequency.

    Returns:
        list: A list of tuples (filename, count), sorted by count in descending order.
              Example: [('index.html', 200), ...]
    """

    # TODO: Implement logic to find top 15 filenames with 404

    # Step 1: Filter only 404 errors
    df_404 = df[df['status'] == 404]

    # Step 2: Count frequency of each filename
    top_15 = (
        df_404['filename']
        .value_counts()
        .head(15)
        .items()
    )

    return list(top_15)


answer5 = top_15_filenames_with_404()
print("Answer 5:")
print(answer5)

Answer 5:
[('index.html', 4694), ('4115.html', 902), ('1611.html', 649), ('5698.xbm', 585), ('710.txt', 408), ('2002.html', 258), ('2177.gif', 193), ('10695.ps', 161), ('6555.html', 153), ('487.gif', 152), ('151.html', 149), ('488.gif', 148), ('3414.gif', 148), ('40.html', 148), ('9678.gif', 142)]


### Q6: Top 15 file extension with 404 responses

In [44]:
def top_15_ext_with_404() -> list[tuple[str, int]]:
    """
    Q6: Top 15 file extensions with 404 responses.

    Objective:
        Find which file extensions generated the most 404 errors.
        Return the top 15 sorted by number of 404s.

    Returns:
        list: A list of tuples (extension, count), sorted by count in descending order.
              Example: [('html', 45), ...]
    """

    # TODO: Implement logic to find top 15 extensions with 404

    df_404 = df[df['status'] == 404]

    # Step 2: Count frequency of extensions
    top_15_exts = (
        df_404['extension']
        .value_counts()
        .head(15)
        .items()
    )

    return list(top_15_exts)


answer6 = top_15_ext_with_404()
print("Answer 6:")
print(answer6)

Answer 6:
[('html', 12142), ('gif', 7202), ('xbm', 824), ('ps', 754), ('jpg', 520), ('txt', 496), ('', 163), ('GIF', 135), ('htm', 107), ('cgi', 77), ('com', 45), ('Z', 41), ('dvi', 40), ('com/', 37), ('ca', 36)]


### Q7: Total bandwidth transferred per day for the month of July 1995

In [46]:
def total_bandwidth_per_day() -> dict[str, int]:
    """
    Q7: Total bandwidth transferred per day for the month of July 1995.

    Objective:
        Sum the number of bytes transferred per day.
        Skip entries where the byte field is missing or '-'.

    Returns:
        dict: A dictionary mapping each date to total bytes transferred.
              Example: {'01-Jul-1995': 123456789, ...}
    """

    # TODO: Implement logic to compute total bandwidth per day

    # Ensure datetime is in correct format
    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
    
    # Drop invalid datetimes
    df_valid = df.dropna(subset=['datetime'])

    # Create a copy of the filtered July 1995 data to avoid SettingWithCopyWarning
    df_july95 = df_valid[
        (df_valid['datetime'].dt.year == 1995) &
        (df_valid['datetime'].dt.month == 7)
    ].copy()

    # Add formatted date string
    df_july95['date_str'] = df_july95['datetime'].dt.strftime('%d-%b-%Y')

    # Group by date and sum bytes
    bandwidth_per_day = df_july95.groupby('date_str')['bytes'].sum().to_dict()

    return bandwidth_per_day


answer7 = total_bandwidth_per_day()
print("Answer 7:")
print(answer7)

Answer 7:
{'01-Jul-1995': 11349799, '02-Jul-1995': 8656918, '03-Jul-1995': 13596612, '04-Jul-1995': 26573988, '05-Jul-1995': 19541225, '06-Jul-1995': 19755015, '07-Jul-1995': 9427822, '08-Jul-1995': 5403491, '09-Jul-1995': 4660556, '10-Jul-1995': 14917754, '11-Jul-1995': 22507207, '12-Jul-1995': 17367065, '13-Jul-1995': 15989234, '14-Jul-1995': 19186430, '15-Jul-1995': 15773233, '16-Jul-1995': 9016378, '17-Jul-1995': 19601338, '18-Jul-1995': 17099761, '19-Jul-1995': 17851725, '20-Jul-1995': 20752623, '21-Jul-1995': 25491617, '22-Jul-1995': 8136259, '23-Jul-1995': 9593870, '24-Jul-1995': 22308265, '25-Jul-1995': 24561635, '26-Jul-1995': 24995540, '27-Jul-1995': 25969995, '28-Jul-1995': 36460693, '29-Jul-1995': 11700624, '30-Jul-1995': 23189598, '31-Jul-1995': 30730715}


### Q8: Hourly request distribution

In [47]:
def hourly_request_distribution() -> dict[int, int]:
    """
    Q8: Hourly request distribution.

    Objective:
        Count the number of requests made during each hour (00 to 23).
        Useful for understanding traffic peaks.

    Returns:
        dict: A dictionary mapping hour (int) to request count.
              Example: {0: 120, 1: 90, ..., 23: 80}
    """

    # TODO: Implement logic for hourly distribution

    # Ensure datetime format
    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
    df_valid = df.dropna(subset=['datetime'])

    # Extract hour and count occurrences
    hourly_counts = (
        df_valid['datetime'].dt.hour
        .value_counts()
        .sort_index()
        .to_dict()
    )

    return hourly_counts


answer8 = hourly_request_distribution()
print("Answer 8:")
print(answer8)

Answer 8:
{0: 11598, 1: 9913, 2: 9403, 3: 8147, 4: 7820, 5: 8283, 6: 9798, 7: 11930, 8: 17351, 9: 21681, 10: 25713, 11: 28665, 12: 26845, 13: 30089, 14: 29792, 15: 28149, 16: 28286, 17: 23312, 18: 17862, 19: 17325, 20: 17488, 21: 15965, 22: 14587, 23: 13613}


### Q9: Top 10 most requested filenames

In [48]:
def top_10_most_requested_filenames() -> list[tuple[str, int]]:
    """
    Q9: Top 10 most requested filenames.

    Objective:
        Identify the most commonly requested URLs (irrespective of status code).

    Returns:
        list: A list of tuples (filename, count), sorted by count in descending order.
                Example: [('index.html', 500), ...]
    """

    # TODO: Implement logic to find top 10 most requested filenames

    top_10 = (
        df['filename']
        .value_counts()
        .head(10)
        .items()
    )
    return list(top_10)


answer9 = top_10_most_requested_filenames()
print("Answer 9:")
print(answer9)

Answer 9:
[('index.html', 139528), ('3.gif', 24006), ('2.gif', 23595), ('4.gif', 8018), ('244.gif', 5148), ('5.html', 5010), ('4097.gif', 4874), ('8870.jpg', 4492), ('6733.gif', 4278), ('8472.gif', 3843)]


### Q10: HTTP response code distribution

In [49]:
def response_code_distribution() -> dict[int, int]:
    """
    Q10: HTTP response code distribution.

    Objective:
        Count how often each HTTP status code appears in the logs.

    Returns:
        dict: A dictionary mapping HTTP status codes (as int) to their frequency.
              Example: {200: 150000, 404: 3000}
    """

    # TODO: Implement logic for response code counts

    return df['status'].value_counts().sort_index().to_dict()


answer10 = response_code_distribution()
print("Answer 10:")
print(answer10)

Answer 10:
{200: 568345, 302: 30295, 304: 97792, 400: 15, 401: 46, 403: 4741, 404: 23517, 500: 42, 501: 43}
